## Modelisation with UNET

In [ ]:
import sys  
sys.path.insert(0, './scripts')

# Import librairies, configs, helper functions
from helpers import *

config = base_config_resized

# config["use_saved_model_path"] = "./model_v4_full_cat_8_large.h5"
config["val_sample_nb"] = 100
config["train_sample_nb"] = 400

config["width"] = 512
config["height"] = 256

config["epoch"] = 4

n = NotebookProcessor(config)

## Model definition

UNET is lorem ipsum

## Model creation

In [ ]:
def conv_block(input_tensor, num_filters, dropout_rate=0.1):  
    x = Conv2D(num_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(input_tensor)  
    x = BatchNormalization()(x)  # Adding batch normalization  
    x = Dropout(dropout_rate)(x)  
    x = Conv2D(num_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)  
    x = BatchNormalization()(x)  # Adding batch normalization  
    return x  
  
def create_model(n_classes, img_width, img_height, img_channels):  
    inputs = Input((img_height, img_width, img_channels))  
  
    # Contraction path  
    c1 = conv_block(inputs, 16)  
    p1 = MaxPooling2D((2, 2))(c1)  
      
    c2 = conv_block(p1, 32)  
    p2 = MaxPooling2D((2, 2))(c2)  
      
    c3 = conv_block(p2, 64)  
    p3 = MaxPooling2D((2, 2))(c3)  
      
    c4 = conv_block(p3, 128)  
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)  
      
    c5 = conv_block(p4, 256, 0.3)  # Increased dropout for the deepest block  
      
    # Expansive path   
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)  
    u6 = concatenate([u6, c4])  
    c6 = conv_block(u6, 128)  
       
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)  
    u7 = concatenate([u7, c3])  
    c7 = conv_block(u7, 64, 0.2)  # Modify dropout rate for specific layers if needed  
      
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)  
    u8 = concatenate([u8, c2])  
    c8 = conv_block(u8, 32)  
       
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)  
    u9 = concatenate([u9, c1])  
    c9 = conv_block(u9, 16)  
       
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)  
       
    model = Model(inputs=[inputs], outputs=[outputs])  
    return model

## Model Stats

In [ ]:
model = create_model(n_classes=config["classes"], img_width=config["width"], img_height=config["height"], img_channels=3)
n.set_model(model)

In [ ]:
n.model_fit()

In [ ]:
# n.model_save("model_delete.h5")

In [ ]:
# display_loss_and_accuracy(n.model_fit_history)

In [ ]:
n.model_predict_with_display_and_accuracy(n.img["val"][3],n.mask["val"][3])

## Model Inference

In [ ]:
n.model_predict_with_display(n.img["val"][3])